In [0]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() #проверяем на засраность

  Running setup.py bdist_wheel for gputil ... - done
  Stored in directory: /root/.cache/pip/wheels/17/0f/04/b79c006972335e35472c0b835ed52bfc0815258d409f560108
Successfully built gputil
  Running setup.py bdist_wheel for humanize ... - \ done
  Stored in directory: /root/.cache/pip/wheels/69/86/6c/f8b8593bc273ec4b0c653d3827f7482bb2001a2781a73b7f44
Successfully built humanize
Gen RAM Free: 13.0 GB  | Proc size: 139.8 MB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


In [0]:
 #!kill -9 -1 #чистим гавно (перезагрузка collab)

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

tcmalloc: large alloc 1073750016 bytes == 0x58cc4000 @  0x7fd76948e2a4 0x594e17 0x626104 0x51190a 0x4f5277 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x4f3338 0x510fb0 0x5119bd 0x4f6070


In [0]:
import torchvision.models as models
import torch.nn as nn
model = models.alexnet(pretrained=True)
num_classes = 20
model.classifier[-1] = nn.Linear(in_features=4096, out_features=num_classes)
nn.init.xavier_uniform(model.classifier[-1].weight)
model

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace)
    (3): Dropout(p=0.5)
    (4): Linear(in_features=4096, out_feature

In [0]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
!ls -la /gdrive/My\ Drive/ML
!cp /gdrive/My\ Drive/ML/VOC2007.tar.gz .
!ls .
!tar -xvf VOC2007.tar.gz

total 2624515
-r-------- 2 root root  753575686 Oct 31 10:02  dogscats_all.tar.gz
-r-------- 2 root root  559404776 Oct 31 12:45  dogscats_train.tar.gz
-rw------- 1 root root 1374517542 Nov  5 16:55  VOC2007.tar.gz
dr-x------ 2 root root       4096 Oct 30 12:48 'Наборы данных'
sample_data  VOC2007.tar.gz
sample_data/VOC2007/VOC2007/
sample_data/VOC2007/VOC2007/test/
sample_data/VOC2007/VOC2007/test/cat/
sample_data/VOC2007/VOC2007/test/cat/000651.jpg
sample_data/VOC2007/VOC2007/test/cat/002003.jpg
sample_data/VOC2007/VOC2007/test/cat/007829.jpg
sample_data/VOC2007/VOC2007/test/cat/007441.jpg
sample_data/VOC2007/VOC2007/test/cat/000114.jpg
sample_data/VOC2007/VOC2007/test/cat/002552.jpg
sample_data/VOC2007/VOC2007/test/cat/009234.jpg
sample_data/VOC2007/VOC2007/test/cat/001389.jpg
sample_data/VOC2007/VOC2007/test/cat/004188.jpg
sample_data/VOC2007/VOC2007/test/cat/001173.jpg
sample_data/VOC2007/VOC2007/test/cat/007332.jpg
sample_data/VOC2007/VOC2007/test/cat/006743.jpg
sample_data/VOC20

In [0]:
from  torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import numpy as np
IMAGE_FOLDER_ROOT = 'sample_data/VOC2007/VOC2007'
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
data_train = ImageFolder(IMAGE_FOLDER_ROOT+'/trainval',transform=transforms.Compose([transforms.Resize(224),transforms.RandomCrop(224),transforms.ToTensor(),normalize]))
data_test = ImageFolder(IMAGE_FOLDER_ROOT+'/test',transform=transforms.Compose([transforms.Resize(224),transforms.RandomCrop(224),transforms.ToTensor(),normalize]))

In [0]:
!pip install Pillow==4.1.1
!pip install PIL
!pip install image
import PIL.Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
train_loader = torch.utils.data.DataLoader(data_train, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(data_test, batch_size=128, shuffle=True)
print(len(data_train), len(data_test))

    100% |████████████████████████████████| 5.7MB 6.5MB/s 
  Found existing installation: Pillow 5.3.0
    Uninstalling Pillow-5.3.0:
      Successfully uninstalled Pillow-5.3.0
  Could not find a version that satisfies the requirement PIL (from versions: )
No matching distribution found for PIL
    100% |████████████████████████████████| 7.3MB 3.7MB/s 
7913 7632


In [0]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)
import torch.optim
import matplotlib.pyplot as plt
import numpy as np

def accuracy(output,labels):
  predictions = torch.argmax(output,dim=1)
  correct = (predictions == labels).sum().cpu().numpy()
  return correct / len(labels)

print(device)

cuda:0


In [0]:
model.eval()

accuracies = []
with torch.no_grad():
  for itr,x in enumerate(test_loader):
    data = x[0]
    labels = x[1]

    data = data.to(device)
    labels = labels.to(device)

    y_pred = model.forward(data)
    acc = accuracy(y_pred,labels)
    accuracies.append(acc)
    #print(device,itr,'/',len(test_loader),acc)


print('accuracy {:.2f}'.format(np.mean(accuracies)))

accuracy 0.08


In [0]:
from torch.optim import lr_scheduler
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

lossf = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum=0.9, nesterov=True)

# Decay LR by a factor of 0.1 every 7 epochs
lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

Parameter containing:
tensor([[-0.0107, -0.0015,  0.0057,  ...,  0.0026, -0.0010, -0.0087],
        [-0.0279,  0.0168, -0.0162,  ..., -0.0079, -0.0049, -0.0219],
        [-0.0112, -0.0184,  0.0073,  ...,  0.0013,  0.0050,  0.0055],
        ...,
        [-0.0027, -0.0146,  0.0057,  ...,  0.0250, -0.0042, -0.0009],
        [ 0.0066,  0.0094,  0.0232,  ...,  0.0060, -0.0029, -0.0042],
        [ 0.0118,  0.0041,  0.0013,  ..., -0.0088, -0.0039,  0.0055]],
       requires_grad=True)


In [0]:
#потестить моментум >0.9 и без него + уменьшение LR (мб динамическое через 3(6???) епоча)
#61 итерация = 1 епоч при батчсайз = 128

model.to(device)
model.train()


for epoch in range(50):
  lr_scheduler.step()
  for itr,x in enumerate(train_loader):
    data = x[0]
    labels = x[1]
    
    optimizer.zero_grad()

    data = data.to(device)
    labels = labels.to(device)
    
    y_pred = model.forward(data)
    
    loss = lossf(y_pred,labels)
    loss.backward()
    optimizer.step()

    print('Iteration {}, train accuracy {:.2f}, loss {:.4f}'.format(itr+epoch*len(train_loader),accuracy(y_pred,labels),loss))

Iteration 0, train accuracy 0.08, loss 4.3172
Iteration 1, train accuracy 0.30, loss 3.2310
Iteration 2, train accuracy 0.17, loss 3.0183
Iteration 3, train accuracy 0.28, loss 2.5400
Iteration 4, train accuracy 0.36, loss 2.2977
Iteration 5, train accuracy 0.34, loss 2.2905
Iteration 6, train accuracy 0.30, loss 2.4545
Iteration 7, train accuracy 0.28, loss 2.4424
Iteration 8, train accuracy 0.40, loss 2.3320
Iteration 9, train accuracy 0.34, loss 2.3852
Iteration 10, train accuracy 0.30, loss 2.2625
Iteration 11, train accuracy 0.34, loss 2.1357
Iteration 12, train accuracy 0.35, loss 2.2457
Iteration 13, train accuracy 0.36, loss 2.1254
Iteration 14, train accuracy 0.41, loss 1.9063
Iteration 15, train accuracy 0.31, loss 2.0515
Iteration 16, train accuracy 0.34, loss 2.0679
Iteration 17, train accuracy 0.39, loss 2.2194
Iteration 18, train accuracy 0.35, loss 2.1462
Iteration 19, train accuracy 0.32, loss 2.0025
Iteration 20, train accuracy 0.37, loss 1.8847
Iteration 21, train acc

In [0]:
model.eval()

accuracies = []
with torch.no_grad():
  for itr,x in enumerate(test_loader):
    data = x[0]
    labels = x[1]

    optimizer.zero_grad()

    data = data.to(device)
    labels = labels.to(device)

    y_pred = model.forward(data)
    accuracies.append(accuracy(y_pred,labels))
    

print('accuracy {:.2f}'.format(np.mean(accuracies)))

accuracy 0.52


In [0]:
torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'model': model,
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, "/gdrive/My Drive/ML/alexnet")